In [ ]:
from cnn import SegmentationModel as net
from DataSet import CamVidDataset
from torchvision import transforms
from d2l import torch as d2l
from tqdm import tqdm
import pandas as pd
import monai
from torchcontrib.optim import SWA
import torch
from torch.utils.data import DataLoader
from torch import nn
import numpy as np
from IOUEval import iouEval
import time
import cv2
import torch.nn.functional as F


In [6]:
val_path   = "C:/Users/asus/Desktop/Video Substraction/DAVIS/ImageSets/480p/val.txt"
model_path = "C:/Users/asus/Desktop/Video Substraction/Output/checkpoints/ESPNetv2_50.pth"
root_path  = 'C:/Users/asus/Desktop/Video Substraction/DAVIS'
val_dataset = CamVidDataset(val_path,root_path)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False,drop_last=True)
toPIL = transforms.ToPILImage()

In [ ]:
def tensor_to_tensor(t):
    dim0,dim1 = t.shape
    res = torch.zeros((dim0,dim1))
    for i in range(dim0):
        for j in range(dim1):
            item = t[i][j]
            if item>0:
                # print(item)
                res[i][j] = 255
            else:
                res[i][j] = 0
    # print(torch.count_nonzero(res).item())
    return res

In [7]:

model = net.EESPNet_Seg(2).cuda()	# 导入网络结构
model = nn.DataParallel(model)
model.load_state_dict(torch.load(model_path)) # 导入网络的参数
devices=d2l.try_all_gpus()
for i, (X, labels) in enumerate(val_loader):
    if isinstance(X, list):
        X = [x.to(devices[0]) for x in X]
    else:
        X = X.to(devices[0])
    gt = labels.long().to(devices[0])
    # print(gt)
    # np.savetxt('C:/Users/asus/Desktop/ESPNetv2-master/mask_pred/'+f"{i}"+'label.txt',labels[0])

    output1,_ = model(X)
    output = F.softmax(output1,dim=1)
    # output = output *255

    # break
    output = output[0][1]
    # output= tensor_to_tensor(output)
    output = output *255
    # print(output)
    output=output.cpu()
    output = output.detach().numpy()
    cv2.imwrite('C:/Users/asus/Desktop/ESPNetv2-master/mask_pred/'+f"{i}"+'.png',output)

    # pic = toPIL(output)
    # pic.save('C:/Users/asus/Desktop/ESPNetv2-master/mask_pred/'+f"{i}"+'.png')
    # gt1 = gt[0].to(dtype=torch.float32)*255
    # # gt1 = tensor_to_tensor(gt1)
    # gt1 = gt1.cpu()
    # # print(gt1.shape)
    # gt1 = gt1.numpy()
    
    # # np.savetxt('C:/Users/asus/Desktop/ESPNetv2-master/mask_pred/'+f"{i}"+'label.txt',gt1)
    # # print(gt1.shape)
    # # pic1 = toPIL(gt1)
    # # pic1.save('C:/Users/asus/Desktop/ESPNetv2-master/mask_pred/'+f"{i}"+'label.png')
    # cv2.imwrite('C:/Users/asus/Desktop/ESPNetv2-master/mask_pred/'+f"{i}"+'label.png',gt1)
    # break
            

In [8]:
dir = 'C:/Users/asus/Desktop/ESPNetv2-master/mask_pred'
dir_o = 'C:/Users/asus/Desktop/ESPNetv2-master/Pred_Mask_480p'

In [13]:
import os
import time
from PIL import Image
def alter(path,object):
  s = os.listdir(path)
  count = 1
  for i in s:
    document = os.path.join(path,i)
    img = Image.open(document)
    out = img.resize((854,480))
    listStr = [str(int(time.time())), str(count)]
    fileName = ''.join(listStr)
    i = i.strip('.png')
    out.save(object+os.sep+'%s.png' % i)
    count = count + 1
alter(dir,dir_o)

In [1]:
import shutil
list_total=[]
list_mask=[]
val0_path = "C:/Users/asus/Desktop/Video Substraction/DAVIS/ImageSets/480p/val0.txt"

In [2]:
with open(val0_path,'r') as f :
        for line in f.readlines():
            line=line.strip('\n')
            list_total.append(line)
        for pair in list_total:
            p = pair.split('   ')
            mask = p[1]

            # list_img.append(img)
            list_mask.append(mask)

In [3]:
original_dir = "C:/Users/asus/Desktop/ESPNetv2-master/Pred_Mask_480p"
output_dir = "C:/Users/asus/Desktop/ESPNetv2-master/Pred_480p"

In [8]:
for i in range(len(list_mask)):
    original_file_path = f"{i+1}.png"
    original_file_path = os.path.join(original_dir,original_file_path)
    # print(original_file_path)
    output_file_name = list_mask[i][1:]
    output_file_path = os.path.join(output_dir,output_file_name)
    dirs = os.path.dirname(output_file_path)
    if not os.path.exists(dirs):
        os.makedirs(dirs)
    os.rename(original_file_path,output_file_path)
    
    

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: 'C:/Users/asus/Desktop/ESPNetv2-master/Pred_Mask_480p\\0.png' -> 'C:/Users/asus/Desktop/ESPNetv2-master/Pred_480p\\blackswan/00000.png '

In [5]:
print(len(list_mask))

1376
